<P id='b'><b><center>Implementation d'un systeme de recommandation dans le domaine de la restauration</center></b></p>

Recuperation de la dataset sous format csv

In [1]:
import pandas as pd
from surprise import Dataset
from surprise import Reader

In [2]:
dataframe=pd.read_csv('food.csv',index_col=False)
dataframe

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."
...,...,...,...,...,...
1132362,116593,72730,2003-12-09,0,Another approach is to start making sauce with...
1132363,583662,386618,2009-09-29,5,These were so delicious! My husband and I tru...
1132364,157126,78003,2008-06-23,5,WOW! Sometimes I don't take the time to rate ...
1132365,53932,78003,2009-01-11,4,Very good! I used regular port as well. The ...


<p><b>Analyse exploratoire<b></p>

In [51]:
dataframe.shape

(1132367, 5)

In [6]:
dataframe.dtypes.value_counts()

int64     3
object    2
dtype: int64

In [7]:
(dataframe.isna().sum()/dataframe.shape[0]).sort_values(ascending=True)

user_id      0.000000
recipe_id    0.000000
date         0.000000
rating       0.000000
review       0.000149
dtype: float64

In [42]:
dataframe.describe()

,user_id,recipe_id,rating
count,1.132367e+06,1.132367e+06,1.132367e+06
mean,1.384291e+08,1.608972e+05,4.411016e+00
std,5.014269e+08,1.303987e+05,1.264752e+00
min,1.533000e+03,3.800000e+01,0.000000e+00
25%,1.354700e+05,5.425700e+04,4.000000e+00
50%,3.309370e+05,1.205470e+05,5.000000e+00
75%,8.045500e+05,2.438520e+05,5.000000e+00
max,2.002373e+09,5.377160e+05,5.000000e+00


In [3]:
data=dataframe[["user_id", "recipe_id", "rating"]] 
data["user_id"].value_counts()

424680        7671
37449         5603
383346        4628
169430        4076
128473        3917
              ... 
244983           1
139446           1
495446           1
113469           1
2001868099       1
Name: user_id, Length: 226570, dtype: int64

In [53]:
data["recipe_id"].value_counts()

2886      1613
27208     1601
89204     1579
39087     1448
67256     1322
          ... 
155682       1
154055       1
252960       1
144013       1
386618       1
Name: recipe_id, Length: 231637, dtype: int64

In [10]:
data["rating"].value_counts()

5    816364
4    187360
0     60847
3     40855
2     14123
1     12818
Name: rating, dtype: int64

In [11]:
print('max recipe_id=',data["recipe_id"].max(),"\n",'max user_id=',data["user_id"].max())

max recipe_id= 537716 
 max user_id= 2002372706


<p><b>Preprocessing</b></p>

In [4]:
from sklearn.model_selection import train_test_split

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [5]:
trainset,testset = train_test_split(data.iloc[0:99999,:],test_size=0.25,random_state=0)

In [6]:
reader = Reader(rating_scale=(0, 5),sep=',')

In [7]:
data=Dataset.load_from_df(trainset,reader)

<p><b>Modele et recherche des meilleurs parametres pour notre modele avec <b><a target=_blank href="https://surprise.readthedocs.io/en/stable/knn_inspired.html"><b>KNNWithMeans<b></a>

In [9]:
from surprise import KNNWithMeans

In [48]:

from surprise.model_selection import GridSearchCV
sim_options = {
    "name": ["msd", "cosine","pearson"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

In [49]:
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

1.2857639598063029
{'sim_options': {'name': 'pearson', 'min_support': 4, 'user_based': True}}


<p><b>Predictions</b></p>

In [8]:
trainsetfull = data.build_full_trainset()

In [10]:
sim_options = {
    "name": "pearson",
    "min_support": 4,
    "user_based": True,
}
modele=KNNWithMeans(sim_options,verbose=False)
modele.fit(trainsetfull)


<p><b>mapping entre les identifiants internes des items generes lors de l'entrainement et ceux fourniss dans la dataset</b></p>

In [9]:
trainset_iids = list(trainsetfull.all_items())
iid_converter = lambda x: trainsetfull.to_raw_iid(x)
trainset_raw_iids = list(map(iid_converter, trainset_iids))

In [10]:
trainset_raw_iids

[156341,
 239061,
 67720,
 151190,
 96346,
 116241,
 67085,
 10080,
 38805,
 400735,
 72614,
 167615,
 32449,
 8650,
 38426,
 51144,
 320743,
 130202,
 141421,
 212395,
 63204,
 359683,
 14684,
 84997,
 226383,
 131044,
 78712,
 24044,
 45539,
 387389,
 390649,
 30965,
 205347,
 134061,
 148874,
 449532,
 157590,
 58247,
 16917,
 144352,
 372406,
 211589,
 101737,
 26110,
 192722,
 359986,
 187989,
 28865,
 166853,
 44572,
 92282,
 89530,
 29255,
 59636,
 73362,
 91798,
 141343,
 231631,
 46019,
 93946,
 80536,
 85706,
 63547,
 80667,
 148172,
 329370,
 33245,
 51997,
 66121,
 127671,
 274902,
 52272,
 3560,
 120680,
 279771,
 166718,
 21135,
 116453,
 371075,
 149356,
 151604,
 212682,
 63889,
 52035,
 258609,
 224035,
 169711,
 90802,
 115129,
 62547,
 216607,
 181085,
 110806,
 8167,
 81339,
 33283,
 96258,
 367376,
 399370,
 110726,
 37413,
 307293,
 211997,
 56103,
 161332,
 3877,
 76233,
 285220,
 456513,
 26526,
 37694,
 287099,
 109173,
 22686,
 70022,
 460206,
 24136,
 62542,


<p><b>mapping entre les identifiants internes des users generes lors de l'entrainement et ceux fourniss dans la dataset</b></p>

In [11]:
trainset_uids = list(trainsetfull.all_users())
uid_converter = lambda x: trainsetfull.to_raw_uid(x)
trainset_raw_uids = list(map(uid_converter, trainset_uids))

In [12]:
trainset_raw_uids

[2897248,
 1481874,
 2000688850,
 2000526979,
 231507,
 89831,
 160981,
 64642,
 255411,
 251917,
 40231,
 28811,
 377039,
 185139,
 464080,
 545825,
 482933,
 113509,
 116796,
 11297,
 382071,
 174096,
 2001640429,
 46104,
 39835,
 670817,
 320625,
 47559,
 37183,
 496803,
 893773,
 85322,
 1687863,
 126440,
 2000225708,
 10769983,
 993709,
 192264,
 333362,
 2000268999,
 470019,
 102058,
 320216,
 2001015054,
 200930,
 1559195,
 28177,
 168921,
 29782,
 440324,
 2001825226,
 583193,
 41738,
 358681,
 511768,
 461283,
 331817,
 326468,
 214008,
 88099,
 2001640614,
 55578,
 62043,
 52448,
 196156,
 528468,
 1049057,
 74281,
 1304051,
 1571500,
 545970,
 705251,
 577249,
 1007198,
 1335303,
 537179,
 349071,
 2002061516,
 32143,
 2001708302,
 1435802,
 2429202,
 461834,
 390420,
 1485433,
 424680,
 280166,
 2001924222,
 39934,
 333017,
 222139,
 469227,
 528617,
 485151,
 216999,
 491095,
 2002211311,
 26075,
 55729,
 829250,
 980739,
 185721,
 908175,
 2693387,
 320843,
 80353,
 22765

<p><b>Enregistrement du modele</b></p>

In [11]:
from surprise.dump import dump

In [13]:
dump(algo=modele,file_name="foodRecommender.pkl",verbose=1)

The dump has been saved as file foodRecommender.pkl
